In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/your_project_folder/'  #change dir to your project folder

In [2]:
df_Alagoas = pd.read_csv('microdadosEnemAlagoas.csv', sep = ';', encoding='ISO 8859-1')

In [3]:
df_Alagoas = df_Alagoas.loc[:, ~df_Alagoas.columns.str.contains('^Unnamed')]

df_Alagoas = df_Alagoas.sample(100)
y = df_Alagoas
X = df_Alagoas


In [4]:
df_Alagoas.sample(5)

,NU_INSCRICAO,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,NU_NOTA_CN,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
52526,180011813228,AL,17.0,0.0,0.0,1.0,0.0,2.0,0.0,372.5,...,A,B,B,A,D,A,A,B,B,A
53445,180012739275,AL,36.0,1.0,1.0,3.0,12.0,1.0,0.0,508.3,...,A,B,A,A,C,A,B,B,A,A
27832,180009587184,AL,20.0,0.0,0.0,2.0,1.0,1.0,0.0,379.9,...,A,B,B,A,C,A,B,B,A,D
43701,180011150990,AL,15.0,0.0,0.0,4.0,0.0,1.0,1.0,413.0,...,A,C,A,B,E,A,B,B,C,B
12317,180009268354,AL,17.0,1.0,0.0,3.0,0.0,2.0,0.0,442.0,...,A,B,A,A,D,A,B,B,B,A


In [5]:
# Leitura dos datasets de treino e teste e criação do df de resposta
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0)
#df_resposta = pd.DataFrame()
print(X.shape, y.shape, X_train.shape, y_train.shape ,X_test.shape, y_test.shape)

(100, 41) (100, 41) (70, 41) (70, 41) (30, 41) (30, 41)


# Variável alvo é a nota de matematica dos participantes

In [6]:
#dataframe que irá receber a resposta dada pelo sistema
df_resposta = pd.DataFrame()

#dataframe que guarda a nota orginal obtida pelo participante
respOriginal_MT = pd.DataFrame()

df_resposta['NU_INSCRICAO'] = X_test['NU_INSCRICAO']

respOriginal_MT =  X_test[ ['NU_INSCRICAO','NU_NOTA_MT']]
print(df_resposta.sample(5) )
print(respOriginal_MT.sample(5) )
y = X_train['NU_NOTA_MT']


       NU_INSCRICAO
60768  180012468778
23722  180009778839
23850  180009646003
62286  180013366413
40565  180010750189
       NU_INSCRICAO  NU_NOTA_MT
12561  180009275102       552.7
60768  180012468778       376.7
58345  180013199623       563.7
36581  180010639630       734.2
44678  180011798657       553.1


In [7]:
df_Alagoas.columns

Index(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
       'IN_TREINEIRO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015',
       'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024',
       'Q025', 'Q026', 'Q027'],
      dtype='object')

In [8]:
var = ['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
       'IN_TREINEIRO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013']
df_Alagoas_1 = df_Alagoas[var]
df_Alagoas_1.sample(5)

,NU_INSCRICAO,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,NU_NOTA_CN,...,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013
53445,180012739275,AL,36.0,1.0,1.0,3.0,12.0,1.0,0.0,508.3,...,D,5.0,D,A,B,C,B,B,B,A
21844,180007417139,AL,16.0,1.0,0.0,3.0,0.0,1.0,1.0,446.0,...,F,4.0,F,A,B,C,A,A,B,A
62782,180013652739,AL,19.0,1.0,0.0,3.0,0.0,1.0,0.0,412.3,...,F,6.0,B,A,B,C,A,A,B,B
34360,180010298229,AL,16.0,0.0,0.0,3.0,0.0,1.0,1.0,514.3,...,D,8.0,C,A,C,D,B,A,B,A
35181,180010518808,AL,18.0,0.0,0.0,1.0,0.0,2.0,0.0,433.4,...,B,4.0,C,A,B,C,A,A,B,A


In [66]:
numeric_features = ['NU_IDADE', 'TP_SEXO','TP_ESTADO_CIVIL', 'TP_COR_RACA','TP_ANO_CONCLUIU', 'TP_ESCOLA',
                    'IN_TREINEIRO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO',  'Q005']

numeric_transformer = Pipeline(   steps=[  ('scaler', StandardScaler())])
#'Q007', 'Q008', 'Q009', 'Q010', 'Q011',
 #      'Q012', 'Q013', 'Q014', 'Q015','Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021',
 #       'Q022', 'Q023', 'Q024', 'Q025', 'Q026', 'Q027'

categorical_features = ['Q001', 'Q002', 'Q003', 'Q004','Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012', 'Q013', 'Q014',
                        'Q015','Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'Q026', 'Q027']

categorical_transformer = Pipeline(steps=[   ('onehot', OneHotEncoder(handle_unknown='ignore'))])

cat1 = ['Q001', 'Q002', 'Q003', 'Q004','Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012', 'Q013', 'Q014']
cat2 = ['Q015','Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'Q026', 'Q027']



X_train1 = X_train[numeric_features + cat1 ]
X_test1 = X_test[numeric_features + cat1 ]


X_train2 = X_train[numeric_features + cat2]
X_test2 = X_test[numeric_features + cat2]

X_train = X_train[numeric_features + categorical_features]
X_test = X_test[numeric_features + categorical_features]



preprocessor = ColumnTransformer(transformers=[
                                        ('num', numeric_transformer, numeric_features),
                                     #  ('cat1', categorical_transformer, cat1),
                                        ('cat', categorical_transformer, categorical_features),
                                        
                                     # ('cat2', categorical_transformer, cat2)
    
])

In [58]:
#print(X_train, X_test)

In [67]:

# execute the data preparation pipeline
    #train_prepared = full_pipeline.fit_transform(train)
    #test_prepared = full_pipeline.fit_transform(test)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
rf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, n_jobs=-1, warm_start=True, random_state=42))
])

#full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
 #                     ('classifier', pipelineRF )])

paramsRF = { 'regressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'regressor__max_depth': [None, 5, 3, 1]}


#clf = GridSearchCV(rf, hyperparameters, cv=10)
clf = GridSearchCV(rf, paramsRF, cv=10)

In [79]:
rf_detector = clf.fit(X_train , y)
print(clf.feature_importances_)
print("model score geral: %.3f" % clf.score(X_train, y))
print("model score geral: %.3f" % clf.score(X_test, y))

model score geral: 0.894


ValueError: Found input variables with inconsistent numbers of samples: [70, 30]

In [77]:
rf_detector1 = clf.fit(X_train1 , y)
print("model score cat1: %.3f" % clf.score(X_train1, y))
print("model score cat1: %.3f" % clf.score(X_test1, y))

C:\Users\aleon\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: 'Q015' is not in list

In [78]:
rf_detector2 = clf.fit(X_train2 , y)
print("model score cat2 : %.3f" % clf.score(X_train2, y))
print("model score geral: %.3f" % clf.score(X_test2, y))

ValueError: 'Q001' is not in list

In [69]:
#print(preprocessor)
#print(X_train.head, X_test.sample(5) )
pred_notas = clf.predict(X_test)

In [76]:

df_resposta['NU_NOTA_MT_Original'] =  respOriginal_MT['NU_NOTA_MT']

In [75]:
pred_notas1 = clf.predict(X_test1)
df_resposta['NU_NOTA_MT_Preditiva1'] = np.around(pred_notas1,1)

C:\Users\aleon\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


ValueError: Input contains NaN

In [63]:
pred_notas2 = clf.predict(X_test2)
df_resposta['NU_NOTA_MT_Preditiva2'] = np.around(pred_notas2,1)

In [64]:
#print(pred_notas)

In [74]:
df_resposta['NU_NOTA_MT_Preditiva'] = np.around(pred_notas,1)




In [72]:
#df_resposta.rename(columns={'NU_NOTA_MT': }, inplace= True)
pd.concat([respOriginal_MT, df_resposta], sort=False)
print( df_resposta)
df_resposta['NU_NOTA_MT_Original'] =  respOriginal_MT['NU_NOTA_MT']

#print( respOriginal_MT, df_resposta)
respOriginal_MT 
df_resposta

       NU_INSCRICAO  NU_NOTA_MT_Preditiva  NU_NOTA_MT_Original  \
60768  180012468778                 464.8                376.7   
61891  180013854922                 520.6                536.9   
23850  180009646003                 532.2                559.6   
43325  180011318700                 520.4                711.0   
34360  180010298229                 499.9                458.8   
23722  180009778839                 523.4                385.3   
59781  180012878527                 466.3                485.6   
44678  180011798657                 531.2                553.1   
49859  180012191069                 564.7                592.2   
35328  180010509499                 462.6                486.5   
51350  180011851592                 496.4                559.2   
4351   180008154280                 500.0                527.0   
36581  180010639630                 554.3                734.2   
12561  180009275102                 489.4                552.7   
36284  180

,NU_INSCRICAO,NU_NOTA_MT_Preditiva,NU_NOTA_MT_Original,NU_NOTA_MT_Preditiva1,NU_NOTA_MT_Preditiva2
60768,180012468778,464.8,376.7,468.3,436.5
61891,180013854922,520.6,536.9,534.4,529.2
23850,180009646003,532.2,559.6,545.2,565.4
43325,180011318700,520.4,711.0,527.3,528.1
34360,180010298229,499.9,458.8,495.6,503.3
23722,180009778839,523.4,385.3,509.0,530.1
59781,180012878527,466.3,485.6,476.6,455.1
44678,180011798657,531.2,553.1,531.3,541.0
49859,180012191069,564.7,592.2,577.7,564.3
35328,180010509499,462.6,486.5,450.7,450.2
